Can you integrate additional data (hierarchical GeoHash/Uber H3/Google S2 encoding for spatial data, properties of the road, additional weather conditions, etc.) from external sources with a reasonable effort?

### check with other datasets

there are missing values, let's check if any of this columns are in the other 2 datasets

In [ ]:
people_df = pd.read_csv('../data/People.csv')
vehicles_df = pd.read_csv('../data/Vehicles.csv')

In [ ]:
for col in set(list(people_df.columns)+list(vehicles_df.columns)):
    if col in missing_values_dict.keys():
        print(f'Column {col} is missing in both People and Vehicles datasets.')

none are (or at least directly)

In [ ]:
people_df.columns, vehicles_df.columns

(Index(['PERSON_ID', 'PERSON_TYPE', 'RD_NO', 'VEHICLE_ID', 'CRASH_DATE', 'CITY',
        'STATE', 'SEX', 'AGE', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED',
        'EJECTION', 'INJURY_CLASSIFICATION', 'DRIVER_ACTION', 'DRIVER_VISION',
        'PHYSICAL_CONDITION', 'BAC_RESULT', 'DAMAGE_CATEGORY', 'DAMAGE'],
       dtype='object'),
 Index(['CRASH_UNIT_ID', 'RD_NO', 'CRASH_DATE', 'UNIT_NO', 'UNIT_TYPE',
        'VEHICLE_ID', 'MAKE', 'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR',
        'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION',
        'MANEUVER', 'OCCUPANT_CNT', 'FIRST_CONTACT_POINT'],
       dtype='object'))

in vehicles we have a feature called TRAVEL_DIRECTION that might contain the same information as STREET_DIRECTION

In [ ]:
#let's join the 2 datasets on rd_no and keeping only street direction and travel direction from the 2
joined_df = crashes_df[['RD_NO', 'STREET_DIRECTION']].merge(
    vehicles_df[['RD_NO', 'TRAVEL_DIRECTION']],
    on='RD_NO',
    how='inner'
)
missing_values = joined_df.isna().sum()

In [ ]:
joined_df

,RD_NO,STREET_DIRECTION,TRAVEL_DIRECTION
0,JC113649,N,S
1,JC113627,N,S
2,JC113627,N,E
3,JC113637,S,N
4,JC113637,S,S
...,...,...,...
460432,HZ164689,N,S
460433,HZ122950,S,S
460434,HZ122950,S,W
460435,JB442550,W,E


In [ ]:
missing_values

RD_NO                   0
STREET_DIRECTION        4
TRAVEL_DIRECTION    10373
dtype: int64

In [ ]:
#mismatch, shape, #shape-#mismatch
(joined_df['STREET_DIRECTION'] != joined_df['TRAVEL_DIRECTION']).sum(), joined_df.shape, joined_df.shape[0]-(joined_df['STREET_DIRECTION'] != joined_df['TRAVEL_DIRECTION']).sum()

(291438, (460437, 3), 168999)

In [ ]:
vehicles_df["TRAVEL_DIRECTION"].isna().sum()

10373

the 2 columns seem to be unrealted to each other:
- other than the fact that 10373 rows values are missing from the travel_direction column,
- the number of mismatch between the total number of rows in the dataset and the number of mismatch (therfore the number of values that match is just ~1/3 of the number of rows, we will have to find this info in another way)

### external integration

need to find street direction since the similar named column in the vehicles dataset didnt yeld any good results

might be able to get LATITUDE, LONGITUDE and LOCATION using the infos in STREET_NAME (and viceversa for the single street name value present)

remember to check if the street name have a null value in them (just check for the frequency of the street names, they should all be different(?), so if there is one repeating it probably is a mock value)

no idea what REPORT_TYPE, BEAT_OF_OCCURRENCE and MOST_SEVERE_INJURY so will need to check the values and the other datasets for something regarding those

### internal integration

since we have CRASH_DATE and DATE_POLICE_NOTIFIED we can find the delta of days that passed between the crash date and the report to the police

BEAT_OF_OCCURRENCE can be found by the latitude and longitude and location data

### dunno

no idea about REPORT_TYPE and MOST_SEVERE_INJURY so will need to check the values and the other datasets for something regarding those